<a href="https://colab.research.google.com/github/Samaroy/Stuff/blob/master/FM_NewGAN_StyleGAN2_Colab_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training StyleGAN2 on Colab

Couple of quick nuances of Google Colab:


1.   When referring to a path on your drive and there is spaces in the folder name for example a folder called "My Folder" - you must refer to this as My\ Folder. the "\" indicates the space is not a break in the code
2.   Try not to play around this the code too much, my understanding is vague and I wont be able to problem solve individual issues
3.   Google Colab has issues. It times out, it has a capacity which based on the number of uses can max out and then randomly just disconnect you, so monitor it often
4. You can't close the tab or your computer while performing this training
5. You must mount your drive every time and select tensorflow 1.x every time. Installing the Repo isnt a must every time but if you are getting errors, reinstalling it sometimes helps. Often its just google colab being temperamental.
6. For a video on this whole process to give you a bit more depth of understanding: https://youtu.be/hv3A62Ojqdg

##Mounting Google Drive


First, connect your Drive to Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Install the repo


In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [ ]:
%tensorflow_version 1.x

##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords).

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.

In [ ]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python dataset_tool.py create_from_images ./datasets/African4 /content/drive/My\ Drive/Machine\ Learning\ Outputs/African4/

/content/drive/My Drive/stylegan2-colab/stylegan2
Loading images from "/content/drive/My Drive/Machine Learning Outputs/African4/"
100% 2020/2020 [11:30<00:00,  2.93it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset=`

This should be the name you used after ".\datasets\" when converting your dataset.

In [ ]:
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=African4 --mirror-augment=true --metrics=none

Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!bash ./easy-install.sh

In [ ]:
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!python run_generator.py generate-images --network=/content/drive/My\ Drive/Machine\ Learning\ Outputs/African\ Pkls/African4.pkl --seeds=1-1000 --truncation-psi=0.7